In [1]:
import sys; sys.path.append(_dh[0].split("knowknow")[0])
from knowknow import *

# README

First, you need to get some data. In accordance with JSTOR's usage policies, I **do not provide any full-text data**. And that's the data you need to use this notebook.
You can obtain your own data by requesting full OCR data packages through JSTOR's [Data for Research](https://www.jstor.org/dfr/) initiative. 

Make sure to read carefully through "User Settings," set the appropriate settings, and run the entire notebook.

This will create a new "database" of counts, which can be recalled by running `my_counts = get_cnt( '<DB_NAME_HERE>' )`.

# User Settings

`database_name` is the name you choose for the final dataset of counts

`zipdir` is the directory which contains the `.zip` files JSTOR provides to you (not included)

`mode` choose between "basic" and "all" mode

1. "basic" mode
    + this mode is not typically faster than `everything`, but it does reduce RAM overhead
        + on ~200k articles the running counters take up more than 16GB RAM
        + to counter this, I first run simple statistics, then rerun this notebook again, filtering based on the descriptive statistics
    + includes `c` counts, the number of citations each document receives
    + includes `c.fj` counts, the number of citations each document receives from each journal's articles
    + includes `c.fy` counts, the number of citations each document receives from each year's articles
    + includes `fj` counts, the number of citations from each journal
    + includes `fj.fy` counts, the number of citations in each journal in each year

2. "all" mode
    + you must run this if you want to run all analyses included in this project
    + includes all counts from `basic` mode
    + includes "term" counts, the counts of tuples appearing in citation contexts (`fj.t` `t` `t.c` `t.fy` `c.t` `t.fy`)
    + includes "author" counts, the number of citations by each author (`a` `a.c` `a.j.y`)
    + includes `c.c`, the cooccurrence network between citations

In [2]:
database_name = 'sociology-jstor'
zipdir = 'G:/My Drive/projects/qualitative analysis of literature/pre 5-12-2020/003 process JSTOR output/RaW dAtA/'
mode = 'all'

use_included_citations_filter = True
use_included_journals_filter = True

NUM_TERMS_TO_KEEP = 1000

In [3]:
# not necessary if you're not filtering based on citations and journals pre-count
included_citations = load_variable("%s.included_citations"%database_name)
included_journals = ['Acta Sociologica', 'Administrative Science Quarterly', 'American Journal of Political Science', 'American Journal of Sociology', 'American Sociological Review', 'Annual Review of Sociology', 'BMS: Bulletin of Sociological Methodology / Bulletin de Méthodologie Sociologique', 'Berkeley Journal of Sociology', 'Contemporary Sociology', 'European Sociological Review', 'Hitotsubashi Journal of Social Studies', 'Humboldt Journal of Social Relations', 'International Journal of Sociology', 'International Journal of Sociology of the Family', 'International Review of Modern Sociology', 'Journal for the Scientific Study of Religion', 'Journal of Health and Social Behavior', 'Journal of Marriage and Family', 'Language in Society', 'Michigan Sociological Review', 'Polish Sociological Review', 'Review of Religious Research', 'Social Forces', 'Social Indicators Research', 'Social Problems', 'Social Psychology Quarterly', 'Sociological Bulletin', 'Sociological Focus', 'Sociological Forum', 'Sociological Methodology', 'Sociological Perspectives', 'Sociological Theory', 'Sociology', 'Sociology of Education', 'Sociology of Religion', 'Symbolic Interaction', 'The American Sociologist', 'The British Journal of Sociology', 'The Canadian Journal of Sociology', 'The Sociological Quarterly', 'Theory and Society']

In [4]:
journal_map = {} # default
journal_map = {
    "Canadian Journal of Sociology / Cahiers canadiens de sociologie": 'The Canadian Journal of Sociology',
    "The Canadian Journal of Sociology / Cahiers canadiens de\n                sociologie": 'The Canadian Journal of Sociology',
    'The Canadian Journal of Sociology / Cahiers canadiens de sociologie': 'The Canadian Journal of Sociology'
}

# imports

In [5]:
# utilities
from nltk import sent_tokenize
from zipfile import ZipFile

# library functions for cleaning and extracting in-text citations from OCR
from cnt_cooc_jstor_lib import (
    extractCitation, getOuterParens, 
    Document, ParseError, 
    clean_metadata
)

# XML parser
from lxml.etree import _ElementTree as ElementTree
from lxml import etree
recovering_parser = etree.XMLParser(recover=True)

# helpers

The following helper function `file_iterator` iterates through all documents inside a list of zipfiles

Each iteration returns:
1. the document DOI
2. the metadata file contents
3. the ocr file contents

In [6]:
def getname(x):
    x = x.split("/")[-1]
    x = re.sub(r'(\.xml|\.txt)','',x)
    return x

def file_iterator(zipfiles):
    from random import shuffle
    
    all_files = []
    for zf in zipfiles:
        archive = ZipFile(zf, 'r')
        files = archive.namelist()
        names = list(set(getname(x) for x in files))
        
        all_files += [(archive,name) for name in names]
        
    shuffle(all_files)
        
    for archive, name in all_files:
        try:
            yield(
                name.split("-")[-1].replace("_", "/"),
                archive.read("metadata/%s.xml" % name),
                archive.read("ocr/%s.txt" % name).decode('utf8')
            )
        except KeyError: # some very few articles don't have both
            continue

`get_page_strings` takes the string contents of an XML file produced by JSTOR. The XML file in question represents the text of a given article. This function cleans it, and splits it into pages

In [7]:
def basic_ocr_cleaning(x):
    # remove multiple spaces in a row
    x = re.sub(r" +", ' ', str(x))
    # remove hyphenations [NOTE this should be updated, with respect to header and footer across pages...]
    x = re.sub(r"([A-Za-z]+)-\s+([A-Za-z]+)", "\g<1>\g<2>", x)
    
    x = x.strip()
    return x

def get_content_string(ocr_string):
    docXml = etree.fromstring(ocr_string, parser=recovering_parser)
    pages = docXml.findall(".//page")

    page_strings = []
    for p in pages:
        if p.text is None:
            continue
        page_strings.append(p.text)

    secs = docXml.findall(".//sec")

    for s in secs:
        if s.text is None:
            continue
        if s.text.strip() == '':
            try_another = etree.tostring(s, encoding='utf8', method='text').decode("utf8").strip()
            #print(try_another)
            if try_another == '':
                continue

            page_strings.append(try_another)
        else:
            page_strings.append(s.text.strip())

    return basic_ocr_cleaning( "\n\n".join(page_strings) )

In [8]:
term_blacklist = set()

def consolidate_terms():
    global term_blacklist
    

    # this is where the filtering occurs
    
    if False:
        # takes terms based on a minimum count, a cutoff
        cutoff = 10
        
        terms = list(cnt_doc['t'].keys())
        counts = np.array([cnt_doc['t'][k] for k in terms])
        
        to_remove = set([terms[int(i)] for i in np.argwhere(counts < cutoff)])
    
    if True:
        # takes the top 5000 terms in terms of yearly count
        sort_them = sorted(cnt_doc['fy.t'], key=lambda x: -cnt_doc['fy.t'][x])
        have_now = set(cnt_doc['t'])
        to_keep = set()
        
        i = 0
        while len(to_keep) < NUM_TERMS_TO_KEEP and i < len(sort_them):
            to_keep.add(sort_them[i][1]) # adds the term, not the year-term pair, to the set
            i += 1
        to_remove = have_now.difference(to_keep)
    
    # so that we never log counts for these again:
    term_blacklist.update(to_remove)

    # the rest of the code is pruning all other term counts for this term in memory
    print("consolidating", len(to_remove), list(to_remove)[:5])
    
    to_prune = ['t','fy.t','fj.t','c.t']
    for tp in to_prune:
        
        whichT = tp.split(".").index('t') # this checks where 't' is in the name of the variable (first or second?)

        print("pruning '%s'..." % tp)

        if tp == 't':
            tydels = to_remove
        else:
            tydels = [x for x in cnt_doc[tp] if x[ whichT ] in to_remove]
            
        print("old size:", len(cnt_doc[tp]))
        for tr in tydels:
            del cnt_doc[tp][tr]
            del cnt_ind[tp][tr]
        print("new size:", len(cnt_doc[tp]))

In [9]:
cits = 0
last_print = 0

def account_for(doc):
    global cits, last_print, mode
    
    # consolidating "terms" counter as I go, to limit RAM overhead
    # I'm only interested in the most common 1000
    if cits - last_print > 100000:
        print("Citation %s" % cits)
        print("Term %s" % len(cnt_doc['t']))
        #print(sample(list(cnt_doc['t']), 10))
        last_print = cits
        consolidate_terms()


    if 'citations' not in doc or not len(doc['citations']):
        #print("No citations", doc['doi'])
        return

    for c in doc['citations']:
        if 'contextPure' not in c:
            raise Exception("no contextPure...")

        sp = c['contextPure'].lower()
        sp = re.sub("[,-`\"'.“]", "", sp) # removing extraneous characters
        sp = re.sub("\s+", " ", sp) # removing extra characters
        sp = sp.split() # splitting into words


        for cited in c['citations']:
            cits += 1
            cnt(doc['year'], 'fy', doc['doi'])

            # citation
            cnt(cited, 'c', doc['doi'])

            # journal
            cnt(doc['journal'], 'fj', doc['doi'])

            # journal year
            cnt((doc['journal'], doc['year']), 'fj.fy', doc['doi'])

            # citation journal
            cnt((cited, doc['journal']), 'c.fj', doc['doi'])

            # citation year
            cnt((cited, doc['year']), 'c.fy', doc['doi'])


        
        
        
        if mode == 'all':

            sp = [x for x in sp if x not in stopwords] # strip stopwords
            tups = set(zip(sp[:-1], sp[1:])) # two-word tuples
            tups = [x for x in tups if x not in term_blacklist]
            

            for cited in c['citations']:
                
                if use_included_citations_filter and (cited not in included_citations):
                    continue
                
                # term features
                for t1 in tups:
                    
                    # cited work, tuple
                    cnt((cited, t1), 'c.t', doc['doi'])

                    # term
                    cnt(t1, 't', doc['doi'])

                    # term year
                    cnt((doc['year'], t1), 'fy.t', doc['doi'])

                    # term journal
                    cnt((doc['journal'], t1), 'fj.t', doc['doi'])

                    if False: # eliminating data I'm not using

                        # author loop
                        for a in doc['authors']:
                            # term author
                            cnt((a, t1), 'a.t', doc['doi'])

                        # term term...
                        for t2 in present_terms:
                            if t1 >= t2:
                                continue

                            if t1 in t2 or t2 in t1:
                                continue

                            # term term
                            cnt((t1,t2), 't.t', doc['doi'])

                # author loop
                for a in doc['authors']:
                    # citation author
                    cnt((a, cited), 'a.c', doc['doi'])

                    # year author journal
                    cnt((a, doc['journal'], doc['year']), 'a.fj.fy', doc['doi'])

                    # author
                    cnt(a, 'a', doc['doi'])

                # add to counters for citation-citation counts
                for cited1 in c['citations']:
                    for cited2 in c['citations']:
                        if cited1 >= cited2:
                            continue

                        cnt(( cited1, cited2 ), 'c.c', doc['doi'])


In [10]:
faulty_years = set()

def citation_iterator(citations):
    for c in citations:
        name = c['names']
        name = name.replace("'s","") # change "Simmel's" to "Simmel"
        
        years = c['year'].split(",")
        for y in years:
            y = y.strip() # get rid of the extra spacing
            y = re.findall(r'[0-9]{4}',y) # this strips out the 'a' in '1995a'
            if not len(y):
                faulty_years.add(c['year'])
                continue
            
            y = y[0]
            yield "%s (%s)" % (name, y)

In [11]:
cnt_ind = defaultdict(lambda:defaultdict(int))
track_doc = defaultdict(lambda:defaultdict(set))
cnt_doc = defaultdict(lambda:defaultdict(int))

def cnt(term, space, doc):
    # it's a set, yo
    track_doc[space][term].add(doc)
    # update cnt_doc
    cnt_doc[space][term] = len(track_doc[space][term])
    # update ind count
    cnt_ind[space][term] += 1

In [12]:
# getting ready for term counting
from nltk.corpus import stopwords as sw
stopwords = set(sw.words('english'))

In [13]:
zipfiles = list(Path(zipdir).glob("*.zip"))

In [14]:
seen = set()

skipped = 0

total_count = Counter()
doc_count = Counter()
pair_count = Counter()

debug = False

for i, (doi, metadata_str, ocr_str) in enumerate( file_iterator(zipfiles) ):

    if i % 1000 == 0:
        print("Document", i, "...", len(cnt_doc['fj'].keys()), "journals so far")

    try:
        drep = clean_metadata( doi, metadata_str )
        
        # sometimes multiple journal names map onto the same journal, for all intents and purposes
        if drep['journal'] in journal_map:
            drep['journal'] = journal_map[drep['journal']]
        
        # only include journals in the list "included_journals"
        if use_included_journals_filter and (drep['journal'] not in included_journals):
            continue
        
        if debug: print("got meta")

        if drep['type'] != 'research-article':
            continue
            
        # some types of titles should be immediately ignored
        def title_looks_researchy(lt):
            lt = lt.lower()
            lt = lt.strip()

            for x in ["book review", 'review essay', 'back matter', 'front matter', 'notes for contributors', 'publication received', 'errata:', 'erratum:']:
                if x in lt:
                    return False

            for x in ["commentary and debate", 'erratum', '']:
                if x == lt:
                    return False

            return True

        lt = drep['title'].lower()
        if not title_looks_researchy(lt):
            continue

        # Don't process the document if there are no authors
        if not len(drep['authors']):
            continue

        drep['content'] = get_content_string(ocr_str)
        
        drep['citations'] = []
        
        # loop through the matching parentheses in the document
        for index, (parenStart, parenContents) in enumerate(getOuterParens(drep['content'])):
            citations = extractCitation(parenContents)
            if not len(citations):
                continue

            citations = list(citation_iterator(citations))
                
            citation = {
                "citations": citations,
                "contextLeft": drep['content'][parenStart-400+1:parenStart+1],
                "contextRight": drep['content'][parenStart + len(parenContents) + 1:parenStart + len(parenContents) + 1 + 100],
                "where": parenStart
            }


            # cut off any stuff before the first space
            first_break_left = re.search(r"[\s\.!\?]+", citation['contextLeft'])
            if first_break_left is not None:
                clean_start_left = citation['contextLeft'][first_break_left.end():]
            else:
                clean_start_left = citation['contextLeft']

            # cut off any stuff after the last space
            last_break_right = list(re.finditer(r"[\s\.!\?]+", citation['contextRight']))
            if len(last_break_right):
                clean_end_right = citation['contextRight'][:last_break_right[-1].start()]
            else:
                clean_end_right = citation['contextRight']

            # we don't want anything more than a sentence
            
            sentence_left = sent_tokenize(clean_start_left)
            if len(sentence_left):
                sentence_left = sentence_left[-1]
            else:
                sentence_left = ""

            sentence_right = sent_tokenize(clean_end_right)[0]
            if len(sentence_right):
                sentence_right = sentence_right[0]
            else:
                sentence_right = ""

            # finally, strip the parentheses from the string
            sentence_left = sentence_left[:-1]
            sentence_right = sentence_right[1:]

            # add the thing in context
            full = sentence_left + "<CITATION>" + sentence_right

            citation['contextPure'] = sentence_left
            #print(full)

            drep['citations'].append(citation)
            
            
            
            
            
        # now that we have all the information we need,
        # we simply need to "count" this document in a few different ways
        account_for(drep)


    except ParseError as e:
        print("parse error...", e.args, doi)

Document 0 ... 0 journals so far
Document 1000 ... 38 journals so far
Document 2000 ... 40 journals so far
Document 3000 ... 40 journals so far
Document 4000 ... 40 journals so far
Document 5000 ... 41 journals so far
Document 6000 ... 41 journals so far
Document 7000 ... 41 journals so far
Document 8000 ... 41 journals so far
Document 9000 ... 41 journals so far
Document 10000 ... 41 journals so far
Citation 100034
Term 435089
consolidating 434089 [('samesex', 'relationships'), ('emphasize', 'sociological'), ('shows', 'contact'), ('specifically', 'claim'), ('hand', 'refers')]
pruning 't'...
old size: 435089
new size: 1000
pruning 'fy.t'...
old size: 533619
new size: 13451
pruning 'fj.t'...
old size: 520810
new size: 10644
pruning 'c.t'...
old size: 943464
new size: 47868
Document 11000 ... 41 journals so far
Document 12000 ... 41 journals so far
Document 13000 ... 41 journals so far
Document 14000 ... 41 journals so far
Document 15000 ... 41 journals so far
Document 16000 ... 41 journ

In [15]:
for k,v in cnt_doc.items():
    print(k, len(v))

fj 41
fy 116
c 461168
fj.fy 1897
c.fj 783972
c.fy 908875
c.t 1486352
t 403677
fy.t 517201
fj.t 498395
a.c 1329587
a.fj.fy 68281
a 37335
c.c 1019768


In [16]:
save_cnt("%s.doc"%database_name, cnt_doc)

Saving sociology-jstor.doc ___ fj
Saving sociology-jstor.doc ___ fy
Saving sociology-jstor.doc ___ c
Saving sociology-jstor.doc ___ fj.fy
Saving sociology-jstor.doc ___ c.fj
Saving sociology-jstor.doc ___ c.fy
Saving sociology-jstor.doc ___ c.t
Saving sociology-jstor.doc ___ t
Saving sociology-jstor.doc ___ fy.t
Saving sociology-jstor.doc ___ fj.t
Saving sociology-jstor.doc ___ a.c
Saving sociology-jstor.doc ___ a.fj.fy
Saving sociology-jstor.doc ___ a
Saving sociology-jstor.doc ___ c.c


In [17]:
save_cnt("%s.ind"%database_name, cnt_ind)

Saving sociology-jstor.ind ___ fy
Saving sociology-jstor.ind ___ c
Saving sociology-jstor.ind ___ fj
Saving sociology-jstor.ind ___ fj.fy
Saving sociology-jstor.ind ___ c.fj
Saving sociology-jstor.ind ___ c.fy
Saving sociology-jstor.ind ___ c.t
Saving sociology-jstor.ind ___ t
Saving sociology-jstor.ind ___ fy.t
Saving sociology-jstor.ind ___ fj.t
Saving sociology-jstor.ind ___ a.c
Saving sociology-jstor.ind ___ a.fj.fy
Saving sociology-jstor.ind ___ a
Saving sociology-jstor.ind ___ c.c
